<a href="https://colab.research.google.com/github/sergak0/AIIJC/blob/main/Lisa/nb_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Зависимости



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output


In [ ]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class BayesModel():
  def __init__()

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')

In [ ]:
data.head()

,Unnamed: 0,texts,y,tokens
0,0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,живо тный лата аристотель выделять организм би...
1,1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,пти цы лата группа теплокровный яйцекладущий п...
2,2,Союз охраны птиц России (СОПР) — общероссийска...,0,союз охрана птица россия сопра общероссийский ...
3,3,Крючковидные отростки рёбер — костяные выросты...,0,крючковидный отросток ребро костяной вырост ве...
4,4,Myxozoa (лат.) — класс или подтип паразитическ...,0,лата класс подтип паразитический беспозвоночны...


In [ ]:
fixed_tokens = []
empty = 0
for  tok in data.tokens:
  aslist = tok.split()
  fixed_tokens.append(aslist)
  empty += aslist.count('')
data.tokens = fixed_tokens

In [ ]:
data.head()

,Unnamed: 0,texts,y,tokens
0,0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,"[живо, тный, лата, аристотель, выделять, орган..."
1,1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,"[пти, цы, лата, группа, теплокровный, яйцеклад..."
2,2,Союз охраны птиц России (СОПР) — общероссийска...,0,"[союз, охрана, птица, россия, сопра, общеросси..."
3,3,Крючковидные отростки рёбер — костяные выросты...,0,"[крючковидный, отросток, ребро, костяной, выро..."
4,4,Myxozoa (лат.) — класс или подтип паразитическ...,0,"[лата, класс, подтип, паразитический, беспозво..."


In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data.head()

,texts,y,tokens
0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,"[живо, тный, лата, аристотель, выделять, орган..."
1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,"[пти, цы, лата, группа, теплокровный, яйцеклад..."
2,Союз охраны птиц России (СОПР) — общероссийска...,0,"[союз, охрана, птица, россия, сопра, общеросси..."
3,Крючковидные отростки рёбер — костяные выросты...,0,"[крючковидный, отросток, ребро, костяной, выро..."
4,Myxozoa (лат.) — класс или подтип паразитическ...,0,"[лата, класс, подтип, паразитический, беспозво..."


In [ ]:
# bigram = gensim.models.Phrases(data.tokens, min_count=10, threshold=100) 
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# bigram_mod.save("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")
processed_docs = make_bigrams(data.tokens)

In [ ]:
glued = []
for i in processed_docs:
  glued.append(' '.join(i))

In [ ]:
glued[1]

'пти цы лата группа теплокровный яйцекладущий позвоночный животное ранг класс обособить группа животное характерный признак покров перо предохранять тело неблагоприятный температура полёт полёт птица вид вторичный передний_конечность форма крыло птица строение орган дыхание пищеварение летать признак птица наличие клюв вид птица регулярный миграция регион земля перемещаться расстояние ареал птица социальный животное общаться визуальный звуковой_сигнал общественный действие коммунальный гнездование охота хищник вид характерный моногамия правило яйцо откладываться гнездо насиживаться член пара птица ухаживать потомство птица населять регион земной_шар антарктида международный союз орнитолог май вид жить птица подвид вид вымереть птица разнообразный класс четвероногий россия вид гнездиться вид вид гнездование страна вид группа позвоночный биология птица природа вид экономический птицеводство отрасль мясо яйцо жир перо вид птица содержаться животное деятельность птица получить уточнить ста

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(glued, data.y, test_size=0.1, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

vect = CountVectorizer()
tfidf = TfidfTransformer()
clf = MultinomialNB()

nb = Pipeline([('vect', vect),
               ('tfidf', tfidf),
               ('clf', clf),
              ])
nb.fit(X_train, Y_train)

%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, Y_test))
print(classification_report(Y_test, y_pred, target_names=['животные', "спорт", "литература", "музыка"]))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
accuracy 0.7485714285714286
              precision    recall  f1-score   support

    животные       0.85      0.93      0.89        44
       спорт       0.79      0.61      0.69        36
  литература       0.74      0.74      0.74        42
      музыка       0.65      0.70      0.67        53

    accuracy                           0.75       175
   macro avg       0.76      0.74      0.75       175
weighted avg       0.75      0.75      0.75       175



In [ ]:
import pickle
pickle.dump(vect, open('/content/drive/MyDrive/aiijc_sber/nb_vect.sav', 'wb'))
pickle.dump(tfidf, open('/content/drive/MyDrive/aiijc_sber/nb_tfidf.sav', 'wb'))
pickle.dump(clf, open('/content/drive/MyDrive/aiijc_sber/nb_clf.sav', 'wb'))

# Usage example

In [ ]:
vect = pickle.load(open('/content/drive/MyDrive/aiijc_sber/nb_vect.sav', 'rb'))
tfidf = pickle.load(open('/content/drive/MyDrive/aiijc_sber/nb_tfidf.sav', 'rb'))
clf = pickle.load(open('/content/drive/MyDrive/aiijc_sber/nb_clf.sav', 'rb'))
nb = Pipeline([('vect', vect),
               ('tfidf', tfidf),
               ('clf', clf),
              ])

In [ ]:
nb.score(X_test, Y_test)

0.7485714285714286